<a href="https://colab.research.google.com/github/healthonrails/annolid/blob/main/docs/tutorials/Train_networks_tutorial_v1.0.1.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Annolid (A segmenation based on mutiple animal tracking package) 


In this tutorial, we will show how to track multiple animals by detection and instance segmentation. The instance segmenatation on images and video is based on [YOLACT](https://github.com/dbolya/yolact).

## Runtime environment in Google Colab
Colab is google's free version of Jupyter notebook. 
This notebook is only tested inside Colab. It might not run in your workstation esp if you are running Windows system. 

### How to use the free GPU?
- Click  **Runtime** > **Change Runtime Type**
- Choose **GPU** > **Save**

## YOLACT(You Only Look At CoefficienTs)
A simple, fully convolutional model for real-time instance segmentation. This is the code for their papers:  
- [YOLACT: Real-time Instance Segmentation](https://arxiv.org/abs/1904.02689)
- [YOLACT++: Better Real-time Instance Segmentation](https://arxiv.org/abs/1912.06218)

## Reference
```
@inproceedings{yolact-iccv2019,
  author    = {Daniel Bolya and Chong Zhou and Fanyi Xiao and Yong Jae Lee},
  title     = {YOLACT: {Real-time} Instance Segmentation},
  booktitle = {ICCV},
  year      = {2019},
}
```
```
@misc{yolact-plus-arxiv2019,
  title         = {YOLACT++: Better Real-time Instance Segmentation},
  author        = {Daniel Bolya and Chong Zhou and Fanyi Xiao and Yong Jae Lee},
  year          = {2019},
  eprint        = {1912.06218},
  archivePrefix = {arXiv},
  primaryClass  = {cs.CV}
}
```
This notebook was inspired and modified from https://colab.research.google.com/drive/1ncRxvmNR-iTtQCscj2UFSGV8ZQX_LN0M. 
https://www.immersivelimit.com/tutorials/yolact-with-google-colab

# Install required packages

In [ ]:
# Cython needs to be installed before pycocotools
!pip install cython
!pip install opencv-python pillow pycocotools matplotlib 
!pip install -U PyYAML

In [ ]:
!nvidia-smi

In [ ]:
# DCNv2 will not work if Pytorch is greater than 1.4.0
!pip install torchvision==0.5.0
!pip install torch==1.4.0

## Clone and install Annolid from GitHub

In [ ]:
# The root folder
%cd /content
# Clone the repo
!git clone --recurse-submodules https://github.com/healthonrails/annolid.git

In [ ]:
# install annolid
%cd /content/annolid/
!pip install -e .

### Please uncomment the following cell and update opencv-python in Colab for the errors as follows.
File "eval.py", line 804, in cleanup_and_exit cv2.destroyAllWindows() cv2.error: OpenCV(4.1.2) /io/opencv/modules/highgui/src/window.cpp:645: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'

In [ ]:
#!pip install -U opencv-python

## DCNv2
If you want to use YOLACT++, compile deformable convolutional layers (from DCNv2).

Note: Make sure you have selected GPU option as discribed in the previous cell.

In [ ]:
# Change to the DCNv2 directory
%cd /content/annolid/annolid/segmentation/yolact/external/DCNv2

# Build DCNv2
!python setup.py build develop

## Download Pretrained Weights
We need to download pre-trained weights for inference. The creator of the GitHub repo shared them on Google Drive. We're going to use a **gdown** to easily access the Drive file from Colab.


In [ ]:
# Make sure we're in the yolact folder
%cd /content/annolid/annolid/segmentation/

# Create a new directory for the pre-trained weights
!mkdir -p /content/annolid/annolid/segmentation/yolact/weights

# Download the file
!gdown --id 1ZPu1YR2UzGHQD0o1rEqy-j5bmEm3lbyP -O ./yolact/weights/yolact_plus_resnet50_54_800000.pth

## Train a model
You can let the model traing for a few hours. Click the stop by for the cell will interrupt the training and save a interrupted model. 
Then you can eval the model based with a save model in the yolact/weigths folder. 

In [ ]:
# Check the GPU availbe for your run
!nvidia-smi

## Download an example dataset

In [ ]:
#https://drive.google.com/file/d/1_kkJ8rlnoMymj0uw8VZkVEuFIMfLDwPh/view?usp=sharing
!gdown --id 1_kkJ8rlnoMymj0uw8VZkVEuFIMfLDwPh -O /content/novelctrl_coco_dataset.zip

In [ ]:
%%shell
#rm -rf /content/novelctrl_coco_dataset//
#rm -rf /content/annolid/annolid/datasets/novelctrl_coco_dataset
unzip /content/novelctrl_coco_dataset.zip -d /content/novelctrl_coco_dataset/
# the following command is only for the demo dataset with relative 
# paths to the dataset in the dataset data.yaml file.
mv /content/novelctrl_coco_dataset/novelctrl_coco_dataset/ /content/annolid/annolid/datasets/


Note please change the path highlighted with ** ** as in the e.g. config data.yaml file to your dataset's location. 
```bash
DATASET:
    name: 'novelctrl'
    train_info: '**/Users/xxxxxx/xxxxxx/novelctrl_coco_dataset/***train/annotations.json'
    train_images: '**/Users/xxxxxx/xxxxxx/novelctrl_coco_dataset/**train'
    valid_info: '**/Users/xxxxxx/xxxxxx/novelctrl_coco_dataset/**valid/annotations.json'
    valid_images: '**/Users/xxxxxx/xxxxxx/novelctrl_coco_dataset/**valid'
    class_names: ['nose', 'left_ear', 'right_ear', 'centroid', 'tail_base', 'tail_end', 'grooming', 'rearing', 'object_investigation', 'left_lateral', 'right_lateral', 'wall_1', 'wall_2', 'wall_3', 'wall_4', 'object_1', 'object_2', 'object_3', 'object_4', 'object_5', 'object_6', 'mouse']
YOLACT:
    name: 'novelctrl'
    dataset: 'dataset_novelctrl_coco'
    max_size: 512

```

## Start Tensorboard

In [ ]:
# Start tensorboard for visualization of training
%cd /content/annolid/annolid/segmentation/yolact/
%load_ext tensorboard
%tensorboard --logdir /content/annolid/runs/logs 

In [ ]:
%cd /content/annolid/annolid/segmentation/yolact/
!python train.py --config=../../datasets/novelctrl_coco_dataset/data.yaml --batch_size=16 --keep_latest_interval=100 # --resume=weights/novelctrl_244_2202_interrupt.pth --start_iter=244

## Mount your GOOGLE Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Save a trained model to your Google Drive
Please replace and put your_model_xxx.pth

In [ ]:
# e.g. 
!cp /content/annolid/annolid/segmentation/yolact/weights/your_model_xxxx.pth /content/drive/My\ Drive

In [ ]:
# optional
#!cp /content/annolid/annolid/segmentation/yolact/weights/resnet50-19c8e357.pth /content/drive/My\ Drive

## Download the original video and test it based on your trained model

In [ ]:
# https://drive.google.com/file/d/1_UDnyYKQplOLMzv1hphk1gVqepmLeyGD/view?usp=sharing
!gdown --id 1_UDnyYKQplOLMzv1hphk1gVqepmLeyGD -O /content/novelctrl.mkv

In [ ]:
!python eval.py --trained_model=weights/novelctrl_xxxx_xxxx.pth --config=../../datasets/novelctrl_coco/data.yaml --score_threshold=0.15 --top_k=9 --video_multiframe=1 --video=/content/novelctrl.mkv:novelctrl_tracked.mkv --display_mask=False --mot

The output results are located at /content/annolid/annolid/segmentation/yolact/results.

tracking_results.csv

xxxx_video_tracked.mp4